In [1]:
import spacy
import pandas as pd
import en_core_web_md
import ast

In [2]:
# load medium English model in case we need to work with vectors
nlp = en_core_web_md.load()

In [3]:
df = pd.read_csv('Neighborhood_Separated_Articles/2014.csv')

In [4]:
black_neighborhoods = ['dorchester', 'roxbury', 'mattapan', 'hyde_park']
white_neighborhoods = ['fenway', 'beacon_hill', 'downtown', 'south_boston', 'east_boston', 'back_bay', 'jamaica_plain',
                      'south_end', 'charlestown', 'brighton', 'allston', 'west_end', 'roslindale', 'north_end',
                      'mission_hill', 'harbor_islands', 'longwood_medical_area', 'west_roxbury']
df = df.fillna("('no article', 'no_id')")
df['dorchester'] = df['dorchester'].apply(ast.literal_eval)
df['roxbury'] = df['roxbury'].apply(ast.literal_eval)
df['mattapan'] = df['mattapan'].apply(ast.literal_eval)
df['hyde_park'] = df['hyde_park'].apply(ast.literal_eval)
df['fenway'] = df['fenway'].apply(ast.literal_eval)
df['beacon_hill'] = df['beacon_hill'].apply(ast.literal_eval)
df['downtown'] = df['downtown'].apply(ast.literal_eval)
df['south_boston'] = df['south_boston'].apply(ast.literal_eval)
df['east_boston'] = df['east_boston'].apply(ast.literal_eval)
df['back_bay'] = df['back_bay'].apply(ast.literal_eval)
df['jamaica_plain'] = df['jamaica_plain'].apply(ast.literal_eval)
df['south_end'] = df['south_end'].apply(ast.literal_eval)
df['charlestown'] = df['charlestown'].apply(ast.literal_eval)
df['brighton'] = df['brighton'].apply(ast.literal_eval)
df['allston'] = df['allston'].apply(ast.literal_eval)
df['west_end'] = df['west_end'].apply(ast.literal_eval)
df['roslindale'] = df['roslindale'].apply(ast.literal_eval)
df['north_end'] = df['north_end'].apply(ast.literal_eval)
df['mission_hill'] = df['mission_hill'].apply(ast.literal_eval)
df['harbor_islands'] = df['harbor_islands'].apply(ast.literal_eval)
df['longwood_medical_area'] = df['longwood_medical_area'].apply(ast.literal_eval)
df['west_roxbury'] = df['west_roxbury'].apply(ast.literal_eval)

In [5]:
spec_chars = ["!",'"',"#","%","&","'","(",")", "*","+",",",
                  "-",".","/",":",";","<", "=",">","?","@","[",
                  "\\","]","^","_", "`","{","|","}","~","–", 
                  "\xc2", "\xa0", "\x80", "\x9c", "\x99", "\x94", 
                  "\xad", "\xe2", "\x9d", "\n"]

df = df.drop(['Unnamed: 0'], axis=1)

#for char in spec_chars:
#    df['text'] = df['text'].str.strip()
#    df['text'] = df['text'].str.replace(char, ' ')
       
# access each column separately
for i in range(len(df.index)):
    for col in df.columns:
        for char in spec_chars:
            try:
                df.loc[i, col][0] = df.loc[i, col][0].str.strip()
                df.loc[i, col][0] = df.loc[i, col][0].str.replace(char, ' ')
            except:
                pass

In [6]:
df.shape

(500, 22)

In [7]:
articles = {'hyde_park': [], 'beacon_hill': [], 'south_boston': [], 'jamaica_plain': [], 'east_boston': [],
                'south_end': [], 'back_bay': [], 'north_end': [], 'west_roxbury': [], 'mission_hill': [],
                'harbor_islands': [], 'west_end': [], 'longwood_medical_area': [],
                'dorchester': [], 'roxbury': [], 'downtown': [], 'fenway': [], 'mattapan': [], 'brighton': [],
                'charlestown': [], 'roslindale': [], 'allston': []}
for sub_neighborhood in articles.keys():
    for i in range(df.shape[0]):
        if type(df.loc[i, sub_neighborhood]) == tuple:
            articles[sub_neighborhood].append((nlp(df.loc[i, sub_neighborhood][0]), df.loc[i, sub_neighborhood][1]))
    print(sub_neighborhood + ' DONE')

hyde_park DONE
beacon_hill DONE
south_boston DONE
jamaica_plain DONE
east_boston DONE
south_end DONE
back_bay DONE
north_end DONE
west_roxbury DONE
mission_hill DONE
harbor_islands DONE
west_end DONE
longwood_medical_area DONE
dorchester DONE
roxbury DONE
downtown DONE
fenway DONE
mattapan DONE
brighton DONE
charlestown DONE
roslindale DONE
allston DONE


In [8]:
articles['dorchester'][0][1]

'2014_5'

In [9]:
people = {'hyde_park': [], 'beacon_hill': [], 'south_boston': [], 'jamaica_plain': [], 'east_boston': [],
                'south_end': [], 'back_bay': [], 'north_end': [], 'west_roxbury': [], 'mission_hill': [],
                'harbor_islands': [], 'west_end': [], 'longwood_medical_area': [],
                'dorchester': [], 'roxbury': [], 'downtown': [], 'fenway': [], 'mattapan': [], 'brighton': [],
                'charlestown': [], 'roslindale': [], 'allston': []}

for sub_neighborhood in articles.keys():
    for (doc, article_id) in articles[sub_neighborhood]:
        for ent in doc.ents:
            if ent.label_ == 'PERSON':
                name = ent[0:2]
                # sentence = ent.sent
                people[sub_neighborhood].append((name, article_id))

In [10]:
for sub_neighborhood in people.keys():
    list1 = people[sub_neighborhood]
    # convert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    people[sub_neighborhood] = unique_list

In [11]:
for sub_neighborhood in people.keys():
    for i in range(len(people[sub_neighborhood])):
        if people[sub_neighborhood][i][0].text.strip() != '':
            temp = people[sub_neighborhood][i][0].text.split()
            if len(temp) > 1:
                people[sub_neighborhood][i] = (temp[-1], people[sub_neighborhood][i][1])
            else:
                people[sub_neighborhood][i] = (temp[0], people[sub_neighborhood][i][1])

In [12]:
people['dorchester'][0]

('Mike', '2014_838')

In [24]:
from ethnicolr import pred_census_ln

In [25]:
# temp = pd.DataFrame(people['fenway'], columns=['last_name', 'article_id'])
# temp = temp.drop(['article_id'], axis=1)
# temp['subneighborhood'] = 'fenway'
# temp.head()
# people['dorchester']

In [32]:
final_df = pd.DataFrame(columns=['last_name', 'subneighborhood'])
subs = white_neighborhoods + black_neighborhoods
subs.remove('longwood_medical_area')
for col in subs:
    temp = pd.DataFrame(people[col], columns=['last_name', 'article_id'])
    temp = temp.drop(['article_id'], axis=1)
    temp['subneighborhood'] = col
    preds = pred_census_ln(temp, 'last_name', 2000)
    final_df = pd.concat([final_df, preds], axis=0)
    print(col + ' DONE')

fenway DONE
beacon_hill DONE
downtown DONE
south_boston DONE
east_boston DONE
back_bay DONE
jamaica_plain DONE
south_end DONE
charlestown DONE
brighton DONE
allston DONE
west_end DONE
roslindale DONE
north_end DONE
mission_hill DONE
harbor_islands DONE
west_roxbury DONE
dorchester DONE
roxbury DONE
mattapan DONE
hyde_park DONE


In [33]:
hen = final_df['race'].value_counts()
pd.DataFrame(hen)

,race
white,26014
hispanic,2837
api,2480
black,216


In [34]:
groups = final_df.groupby(final_df.subneighborhood)
race_df = pd.DataFrame(columns=subs)
races = final_df.race.unique()
for col in subs:
    group = groups.get_group(col)
    race_df[col] = group.race.value_counts()

In [35]:
race_df = race_df.fillna(0.0)
race_df

,fenway,beacon_hill,downtown,south_boston,east_boston,back_bay,jamaica_plain,south_end,charlestown,brighton,...,west_end,roslindale,north_end,mission_hill,harbor_islands,west_roxbury,dorchester,roxbury,mattapan,hyde_park
white,353,790,1041,2255,456,1257,907,718,189,244.0,...,103,492,147.0,459,11.0,858,9339,3540,703,1916
hispanic,184,15,102,151,266,27,61,134,12,29.0,...,2,71,19.0,18,0.0,32,1040,171,53,443
api,66,78,91,178,38,85,82,84,19,15.0,...,9,42,14.0,33,0.0,92,883,312,67,232
black,2,2,1,16,10,9,4,34,1,0.0,...,2,1,0.0,3,0.0,7,58,45,3,10


In [36]:
for i in range(len(subs)):
    tot = race_df[subs[i]].sum()
    for j in race_df.index:
        race_df.loc[j, [subs[i]]] = (race_df.loc[j, [subs[i]]] / tot) * 100

In [37]:
race_df

,fenway,beacon_hill,downtown,south_boston,east_boston,back_bay,jamaica_plain,south_end,charlestown,brighton,...,west_end,roslindale,north_end,mission_hill,harbor_islands,west_roxbury,dorchester,roxbury,mattapan,hyde_park
white,58.347107,89.265537,84.291498,86.730769,59.220779,91.219158,86.053131,74.020619,85.520362,84.722222,...,88.793103,81.188119,81.666667,89.473684,100.0,86.754297,82.500000,87.020649,85.108959,73.663975
hispanic,30.413223,1.694915,8.259109,5.807692,34.545455,1.959361,5.787476,13.814433,5.429864,10.069444,...,1.724138,11.716172,10.555556,3.508772,0.0,3.235592,9.187279,4.203540,6.416465,17.031911
api,10.909091,8.813559,7.368421,6.846154,4.935065,6.168360,7.779886,8.659794,8.597285,5.208333,...,7.758621,6.930693,7.777778,6.432749,0.0,9.302326,7.800353,7.669617,8.111380,8.919646
black,0.330579,0.225989,0.080972,0.615385,1.298701,0.653120,0.379507,3.505155,0.452489,0.000000,...,1.724138,0.165017,0.000000,0.584795,0.0,0.707786,0.512367,1.106195,0.363196,0.384468


In [38]:
#race_df.index = ['white', 'hispanic', 'black', 'asian']
race_df

,fenway,beacon_hill,downtown,south_boston,east_boston,back_bay,jamaica_plain,south_end,charlestown,brighton,...,west_end,roslindale,north_end,mission_hill,harbor_islands,west_roxbury,dorchester,roxbury,mattapan,hyde_park
white,58.347107,89.265537,84.291498,86.730769,59.220779,91.219158,86.053131,74.020619,85.520362,84.722222,...,88.793103,81.188119,81.666667,89.473684,100.0,86.754297,82.500000,87.020649,85.108959,73.663975
hispanic,30.413223,1.694915,8.259109,5.807692,34.545455,1.959361,5.787476,13.814433,5.429864,10.069444,...,1.724138,11.716172,10.555556,3.508772,0.0,3.235592,9.187279,4.203540,6.416465,17.031911
api,10.909091,8.813559,7.368421,6.846154,4.935065,6.168360,7.779886,8.659794,8.597285,5.208333,...,7.758621,6.930693,7.777778,6.432749,0.0,9.302326,7.800353,7.669617,8.111380,8.919646
black,0.330579,0.225989,0.080972,0.615385,1.298701,0.653120,0.379507,3.505155,0.452489,0.000000,...,1.724138,0.165017,0.000000,0.584795,0.0,0.707786,0.512367,1.106195,0.363196,0.384468


In [39]:
race_df.to_csv('Census2000_Last_Name_Race_Representation_by_Neighborhood/2014_representation.csv')

In [19]:
#agg_df = pd.DataFrame(columns=subs, index=['white', 'black', 'api', 'hispanic'])

In [20]:
#agg_df = agg_df.fillna(0.0)

In [21]:
#pred_races = race_df.index
#for sub in subs:
#    for each in pred_races:
#        temp = race_df.loc[each][sub]
#        if each == 'GreaterEuropean,WestEuropean,Hispanic':
#            agg_df.loc['hispanic'][sub] += temp
#        elif each == 'Asian,GreaterEastAsian,EastAsian' or each == 'Asian,IndianSubContinent' or each == 'Asian,GreaterEastAsian,Japanese':
#            agg_df.loc['api'][sub] += temp
#        elif each == 'GreaterAfrican,Muslim' or each == 'GreaterAfrican,Africans':
#            agg_df.loc['black'][sub] += temp
#        else:
#            agg_df.loc['white'][sub] += temp

In [22]:
#agg_df.to_csv('Wiki_Full_Name_Race_Representation_by_Neighborhood/2014_representation.csv')

In [23]:
#agg_df